In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.segmentation import clear_border
from skimage import measure
from skimage.measure import label,regionprops
from scipy import ndimage as ndi
from scipy.ndimage import measurements, center_of_mass, binary_dilation, zoom
import plotly.graph_objects as go

Open data

In [ ]:
import os
import zipfile
import numpy as np

zipfile_path = r'../Data/CT_scan.npy.zip'
filepath = r'../Data/CT_scan.npy'

# Check if the file exists
if not os.path.exists(filepath):
    # If not, extract it from the zip file
    with zipfile.ZipFile(zipfile_path, 'r') as zip_ref:
        zip_ref.extractall(os.path.dirname(filepath))

# Load the numpy file
img = np.load(filepath)

In [ ]:
img.shape

Plot slice of the image:

In [ ]:
plt.figure(figsize=(8,8))
plt.pcolormesh(img[178], cmap='Greys_r')

What do these units mean? The pixel values of this CT scan are expressed in Hounsfield Units

$$HU(x,y) \equiv 1000 \cdot \frac{\mu(x,y) - \mu_{\text{water}}}{\mu_{\text{water}}-\mu_\text{air}}$$

where $\mu$ is the linear attenuation coefficient of the material. The linear attenuation coefficient is defined based on how the intensity of a photon beam decays as it passes a distance $x$ through a material $I=I_0e^{-\mu x}$. Note that $\mu$ depends on the energy of the photon beam, and in a CT scan photons usually have energies $\approx 100$keV. Here are typical HU values:

# Modify data

The first thing to note is that air is signifcantly less HU than other substances in the body, so we apply a so-called "threshhold" mask. Lets use -320 HU as the lower limit:

In [ ]:
mask = img < -320

Plot the mask

In [ ]:
plt.pcolormesh(mask[180])

Next we can use the `clear_border` function to remove the outer border:

In [ ]:
mask.shape

In [ ]:
mask = np.vectorize(clear_border, signature='(n,m)->(n,m)')(mask)
plt.pcolormesh(mask[170])

Now we'll give each seperate volume a different integer value using the `label` function

In [ ]:
mask.shape

In [ ]:
mask_labeled = np.vectorize(label, signature='(n,m)->(n,m)')(mask)
plt.pcolormesh(mask_labeled[170])
plt.colorbar()

Now for something a little non-intuitive. We want to keep the three largest areas for each slice of the image. Why would we want to do this if we only want to keep the two lungs?

* In some slices one of the lungs could be larger than the table
* In some slices one of the lungs could be smaller than the table

If we only take the largest two slices, we might end up taking the table and one of the lungs. So for now lets take the top 3. The function below is designed to operate on a single slice of the 3D image:

In [ ]:
slc = mask_labeled[170]

In [ ]:
rps = regionprops(slc)

In [ ]:
areas = [r.area for r in rps]

In [ ]:
areas

In [ ]:
np.argsort(areas)[::-1]

In [ ]:
slc = mask_labeled[170]
rps = regionprops(slc)
areas = [r.area for r in rps]
idxs = np.argsort(areas)[::-1] # we want largest to smallest

Only consider the 3 largest areas (iterating theough `idxs`). Add these areas to a new slice called `new_slc`:

In [ ]:
new_slc = np.zeros_like(slc)

In [ ]:
new_slc = np.zeros_like(slc)
for i in idxs[:3]:
    new_slc[tuple(rps[i].coords.T)] = i+1

Plot 

In [ ]:
plt.pcolormesh(new_slc)

Now lets automate this for all slices in our 3D image:

In [ ]:
def keep_top_3(slc):
    new_slc = np.zeros_like(slc)
    rps = regionprops(slc)
    areas = [r.area for r in rps]
    idxs = np.argsort(areas)[::-1]
    for i in idxs[:3]:
        new_slc[tuple(rps[i].coords.T)] = i+1
    return new_slc

In [ ]:
mask_labeled = np.vectorize(keep_top_3, signature='(n,m)->(n,m)')(mask_labeled)

In [ ]:
plt.pcolormesh(mask_labeled[165])

Now lets fill in any small holes in the lungs:

In [ ]:
mask = mask_labeled > 0

In [ ]:
mask.shape

In [ ]:
mask.shape

In [ ]:
mask = mask_labeled > 0
mask = np.vectorize(ndi.binary_fill_holes, signature='(n,m)->(n,m)')(mask)

In [ ]:
plt.pcolormesh(mask[170])

In some slices, the trachea is kind of annoying and we need to remove it:

In [ ]:
plt.pcolormesh(mask[-50])

In a 512x512 image, the trachea typically takes up less than 0.69% of the area. We can delete all regions that have any area smaller than this percentage:

In [ ]:
mask[-50]

In [ ]:
labels = label(mask[-50],connectivity=1,background=0)

In [ ]:
plt.pcolormesh(labels)

In [ ]:
rps = regionprops(labels)

In [ ]:
rps

In [ ]:
areas = np.array([r.area for r in rps])

In [ ]:
areas

In [ ]:
np.where(areas/512**2 < 0.0069)

In [ ]:
def remove_trachea(slc, c=0.0069):
    new_slc = slc.copy()
    labels = label(slc,connectivity=1,background=0)
    rps = regionprops(labels)
    areas = np.array([r.area for r in rps])
    idxs = np.where(areas/512**2 < c)[0]
    for i in idxs:
        new_slc[tuple(rps[i].coords.T)] = 0
    return new_slc

In [ ]:
mask = np.vectorize(remove_trachea, signature='(n,m)->(n,m)')(mask)

Now the trachea is removed in the slice we were considering:

In [ ]:
plt.pcolormesh(mask[170])

Finally, its time to remove the table. Note that the center of mass of the table is always lower than the two lungs. As such, we simply need to delete the volume with the lowest center of mass in $y$ to delete the table:

In [ ]:
labels = label(mask[170], background=0)

Plot a slice of the labels with the masks:

In [ ]:
plt.pcolormesh(labels)
plt.colorbar()

Compute the center of masses in this slice:

In [ ]:
center_of_mass(labels==3)[0]

In [ ]:
def delete_table(slc):
    new_slc = slc.copy()
    labels = label(slc, background=0)
    idxs = np.unique(labels)[1:]
    COM_ys = np.array([center_of_mass(labels==i)[0] for i in idxs])
    for idx, COM_y in zip(idxs, COM_ys):
        if (COM_y < 0.3*slc.shape[0]):
            new_slc[labels==idx] = 0
        elif (COM_y > 0.6*slc.shape[0]):
            new_slc[labels==idx] = 0
    return new_slc

In [ ]:
mask_new = np.vectorize(delete_table, signature='(n,m)->(n,m)')(mask)

In [ ]:
plt.pcolormesh(mask_new[167])

Finally, we can expand the area of the lungs a little bit by growing the border. For this, we can use the `binary_dilation` function:

In [ ]:
mask_new = binary_dilation(mask_new, iterations=5)

In [ ]:
plt.figure(figsize=(8,8))
plt.pcolormesh(mask_new[170], cmap='brg')

Lets plot the full 3D image in plotly and create an interactive plot:

* First decrease the resolution a little bit:

In [ ]:
im = zoom(1*(mask_new), (0.4,0.4,0.4))

Get arrays of $x$, $y$, and $z$. In a CT scan, the difference between pixels in the z direction is about 4 times bigger than in the $x$ and $y$ directions:

In [ ]:
z, y, x = [np.arange(i) for i in im.shape]
z*=4

Create a meshgrid:

In [ ]:
X,Y,Z = np.meshgrid(x,y,z, indexing='ij')

Create a 3D plotly plot

In [ ]:
fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=np.transpose(im,(1,2,0)).flatten(),
    isomin=0.1,
    opacity=0.1, # needs to be small to see through all surfaces
    surface_count=17, # needs to be a large number for good volume rendering
    ))
fig.write_html("test.html")

In [ ]:
img_new = mask_new * img

In [ ]:
plt.pcolormesh(img_new[170])